# Searching
Search algorithms can be classified in a number of different ways:
- static or dynamic, i.e., inserts and deletes are interleaved with searching
- is it worth the computational cost to preprocess the data so as to speed up subsequent queries?
- are there statistical properties of the data that can be exploited?
- should we operate on the data directly or transform it?   

Section covers static data stored in a sorted array


In [2]:
import bisect
import collections
import math
import random
from typing import Callable, List, Tuple

from utils import run_tests

## Binary Search
The idea is to eliminate half the keys under consideration by keeping keys in sorted order. If the search key is not equal to the middle element of the array, one of the two sets of keys to the left or righ of the middle element can be eliminated from further considerations. 

Time complexity is $T(n) = T(n/2) + c = O(\log n)$   
$O(n)$ if array is unsorted   
One disadvantage is that it takes $O(n\log n)$ time to sort but this is not an issue if there are many searches to be performed on the array

## Tips
- **Binary search** is an effective search tool. It is applicable to more than just searching in **sorted arrays**, e.g., it can be used to sarch an **interval of real numbers or integers**
- If your solution uses sorting, and the computation performed after sorting is faster than sorting, e.g. $O(n)$ or $O(\log n)$, **look for solutions that do not perform a complete sort**
- Consider **time/space tradeoffs**, such as making multiple passes through the data. 

In [3]:
def binary_search(x: int, A: List[int]) -> int:
    L, U = 0, len(A) - 1    # lower and upper bounds respectively


    while L <= U:
        # this can cause overflow issues
        # could replace with M = L + (U - L) // 2
        M = (L + U) // 2        # midpoint 

        if A[M] == x:
            return M 
    
        # search upper half
        elif A[M] < x:     
            L = M + 1

        # search lower half
        else: 
            U = M - 1         
    
    # value not in array
    return -1


A = list(range(10))
print(binary_search(5, A))
print(binary_search(20, A))
A = [1, 4, 5, 10, 11, 30, 45]
print(binary_search(11, A))

5
-1
4


In [4]:
def binary_search_recursive(x: int, A: List[int]) -> int:
    L, U = 0, len(A) - 1    # lower and upper bounds respectively

    M = (L + U) // 2        # midpoint 

    if A[M] == x:
        return M 

    # search upper half
    elif A[M] < x:     
        L = M + 1
        if L > U:
            return -1
        else:
            binary_search(x, A[L:])

    # search lower half
    else: 
        U = M - 1      
        if U > L:
            return -1
        else:
            binary_search(x, A[:U+1])
    


A = list(range(10))
print(binary_search(5, A))
print(binary_search(20, A))
A = [1, 4, 5, 10, 11, 30, 45]
print(binary_search(11, A))

5
-1
4


## Libraries
bisect module

In [5]:
# binary search algorithm
# Note: returns index of where to insert element, i.e., one plus above solution
print('bisect search:', bisect.bisect(A, 11))

# find first element not less than target
print('bisect left:', bisect.bisect_left(A, 11))

# find first element that is greater than target
print('bisect right:', bisect.bisect_right(A, 11))

bisect search: 5
bisect left: 4
bisect right: 5


In [6]:
Student = collections.namedtuple('Student', ('name', 'gpa'))

def comp_gpa(student: Student) -> Tuple[float, str]:
    return (-student.gpa, student.name)

def search_student(students: List[Student], target: Student, comp_gpa: Callable[[Student], Tuple[float, str]]):
    i = bisect.bisect_left([comp_gpa(s) for s in students], comp_gpa(target))
    return 0 <= i < len(students) and students[i] == target

### 11.1: Return *first* index equal to specified element

In [7]:
def search_first_k(A: List[int], x: int) -> int:
    L, U, result = 0, len(A) - 1, -1

    while L <= U:
        M = (L + U) // 2
        if A[M] == x:
            result = M 
            U = M - 1    # nothing to right of M can be solution
        elif A[M] < x:   # search upper half
            L = M + 1
        else:            # search lower half
            U = M - 1
    
    return result

A = [-14, -10, 2, 108, 108, 243, 285, 285, 285, 401]
assert search_first_k(A, 108) == 3
assert search_first_k(A, 285) == 6

Time complexity $(\log n)$ since each iteration reduces the size of the candidate set by half

#### Variant: Find *first* occurence of element greater than key

In [8]:
def search_first_greater_k(A: List[int], x: int) -> int:
    L, U, result = 0, len(A) - 1, -1

    while L <= U:
        M = (L + U) // 2
        if A[M] <= x:   # search upper half
            L = M + 1
        # A[m] > x
        else:            # search lower half
            result = M 
            U = M - 1    # nothing to right of M can be solution
    
    return result

assert search_first_greater_k(A, 108) == 5
assert search_first_greater_k(A, 285) == 9
assert search_first_greater_k(A, -13) == 1

#### Variant: Unsorted Array

#### Variant: Return interval enclosing key, i.e. all values equal to key

In [9]:
def search_all_k(A: List[int], x: int) -> int:
    L, U, result = 0, len(A) - 1, [-1, -1]

    while L <= U:
        M = (L + U) // 2
        if A[M] == x:
            
            start_idx = end_idx = M

            # search to left of M
            while start_idx - 1 >= 0 and A[start_idx - 1] == x:
                start_idx -= 1
            
            # search to right of M
            while end_idx + 1 <= len(A) and A[end_idx + 1] == x:
                end_idx += 1

            result[0], result[1] = start_idx, end_idx
            return result

        elif A[M] < x:   # search upper half
            L = M + 1
        else:            # search lower half
            U = M - 1
    
    return result

A = [1, 2, 2, 4, 4, 4, 7, 11, 11, 13]
assert search_all_k(A, 11) == [7, 8]
assert search_all_k(A, 2) == [1, 2]
assert search_all_k(A, 4) == [3, 5]
assert search_all_k(A, 7) == [6, 6]
assert search_all_k(A, 5) == [-1, -1]

$O(\log n + m)$ time complexity where $m<n$ and $m$ is the number of elements in the list equal to the key. $O(n)$ worst-case scenario

In [10]:
def search_all_k(A: List[int], x: int) -> int:
    result = [-1, -1]

    # search for first value in list equal to key
    lower_bound = search_first_k(A, x)
    if lower_bound == -1:
        return result
    else:
        result[0] = lower_bound
    
    # search for first value in list greater than key
    upper_bound = search_first_greater_k(A, x)
    result[1] = upper_bound - 1

    return result

A = [1, 2, 2, 4, 4, 4, 7, 11, 11, 13]
assert search_all_k(A, 11) == [7, 8]
assert search_all_k(A, 2) == [1, 2]
assert search_all_k(A, 4) == [3, 5]
assert search_all_k(A, 7) == [6, 6]
assert search_all_k(A, 5) == [-1, -1]

$O(\log n)$ time complexity

#### Variant: Write a program which tests if *p* is a prefix of a string in an array of sorted strings

### 11.2: Search a sorted array of distinct integers for entry equal to its index

In [11]:
def search_entry_equal_to_index(A: List[int]) -> int:
    L, U = 0, len(A) - 1

    while L <= U:
        M = (L + U) // 2

        if A[M] == M:
            return M
        elif A[M] > M: # value greater than index so know all values to right will be greater than index
            U = M - 1
        else:          # value less than index
            L = M + 1
    return -1

A = [-2, 0, 2, 3, 6, 7, 9]
print(search_entry_equal_to_index(A))
A = [-2, 0, 2, 4, 5, 9]
print(search_entry_equal_to_index(A))

3
2


Time complexity $(\log n)$

#### Variant: Search a sorted array of integers with duplicates for entry equal to its index
- if $A[M] < M$ search to right of M
- if $A[M] > len(A)$ search to right
- if $A[M] > M and A[m] < len(A)$ need to search both sides

### 11.3: Search a cyclically sorted array for minimum
an array is cyclically sorted if it is possible to cyclically shift its entries s.t. the array is sorted

e.g. $[70, 85, 100, 120, 20, 30, 34, 50, 68]$

In [12]:
# assume unique integers in array
def search_min_cyclic_array(A: List[int]) -> int:
    L, U = 0, len(A) - 1

    while L < U:
        M = (L + U) // 2
        if A[M] < A[U]:   # minimum cannot be in A[M+1, U]
            U = M
        # A[M] > A[U]
        else:             # minimum cannot be in A[:, M]
            L = M + 1
    
    # L == U
    return L

assert search_min_cyclic_array([70, 85, 100, 120, 20, 30, 34, 50, 68]) == 4

Time complexity $(\log n)$.
Time complexity is $(n)$ if elements can repeat

### 11.3.A: Variant: Find maximum in strictly ascending then strictly descending array
e.g. $[1, 2, 3, 5, 7, 8, 4, 2]$

In [13]:
def search_max_asc_desc_array(A: List[int]) -> int:
    L, U = 0, len(A) - 1

    while L < U:
        M = (L + U) // 2
        if A[M] < A[M+1]:   # still ascending to the right of M
            L = M + 1
        # A[M] > A[M+1]     # descending to right of M
        else:
            U = M
    
    # L == U 
    return L

assert search_max_asc_desc_array([1, 2, 3, 5, 7, 8, 4, 2]) == 5
assert search_max_asc_desc_array([1, 2, 3, 5, 7, 8, 4, 2, 1]) == 5
assert search_max_asc_desc_array([1, 100, 95, 80]) == 1

$O(\log n)$ time complexity

#### Variant: Search a cyclically sorted array for key

### 11.4: Compute the integer squre root
e.g. $\sqrt{16} = 4$  and $\sqrt{300} = 17$ because $17^2 = 289 < 300$ and $18^2 = 324 > 300$

In [14]:
def integer_square_root(x: int) -> int:
    L, U = 0, x 

    # careful with the bounds especially for when integer square exists
    while L <= U:
        M = (L + U) // 2
        square = M * M

        if square > x:
            U = M - 1
        else:
            L = M + 1
   

    return L - 1

assert integer_square_root(16) == 4
assert integer_square_root(100) == 10
assert integer_square_root(120) == 10
assert integer_square_root(300) == 17

$O(\log n)$ time complexity

### 11.5 Compute Square Root

In [15]:
def square_root(x: int) -> float:

    # decide the search range according to x's value relative to 1.0
    L, U = (1.0, x) if x >= 1 else (x, 1.0)

    while not math.isclose(L, U):
        M = (L + U) * 0.5
        square = M * M
        if square < x:
            L = M 
        else:
            U = M 

    return L

print(square_root(100))
print(square_root(16))
print(square_root(300))

9.999999997380655
3.999999999301508
17.320508067845367


Time complexity is $O(\log \frac{x}{s})$ where $s$ is the tolerance

## Generalized Search
Do not use the binary search principle. Focus on tradeoffs between RAM and computation time, avoid wasted comparisions when searching for the minimum and maximum simultaneously, use randomization to perform elimination efficiently, use bit-level manipulation to identify missing elements, ect.

### 11.6: Search in a 2D Sorted Array
A 2D array is sorted if both its rows and column are non-decreasing

In [16]:
def search_k_2d(A: List[List[int]], k) -> True:

    # start in upper-right corner
    row, col = 0, len(A[0]) - 1

    while row < len(A) and col >= 0:

        if A[row][col] == k:
            return True
        elif A[row][col] < k:  # eliminate row
            row += 1
        else:                  # A[row][col] > k
            col -= 1           # eliminate column
    
    # key not found
    return False


A = [
        [-1, 2, 4, 4, 6],
        [1, 5, 5, 9, 21],
        [3, 6, 6, 9, 22],
        [3, 6, 8, 10, 24],
        [6, 8, 9, 12, 25],
        [8, 10, 12, 13, 40]
    ]

assert search_k_2d(A, 8) == True
assert search_k_2d(A, 7) == False
assert search_k_2d(A, 6) == True
assert search_k_2d(A, 40) == True
assert search_k_2d(A, -1) == True

Time complexity is $O(m + n)$ since eliminate row or column at every iteration

### 11.7: Find the Min and Max Simultaneously

In [17]:
MinMax = collections.namedtuple('MinMax', ('min', 'max'))

def find_min_max(A: List[int]) -> MinMax:
    def min_max_helper(a:int, b:int) -> MinMax:
        return MinMax(a, b) if a < b else MinMax(b, a)

    if len(A) == 1:
        return MinMax(A[0], A[1])
    
    # initalize first two elements as min-max
    global_min_max = min_max_helper(A[0], A[1])

    # compare pairs of integers with each other then update overall min-max
    for i in range(2, len(A)-1, 2):                                 # careful with bounds on odd
        local_min_max = min_max_helper(A[i], A[i+1])
        global_min_max = MinMax(
            min([global_min_max.min, local_min_max.min]), 
            max([global_min_max.max, local_min_max.max])
        )

    # if odd number of elements, need to check last element
    if len(A) % 2 == 1:
        global_min_max =  MinMax(
                                min([global_min_max.min, A[-1]]),
                                max([global_min_max.max, A[-1]])
                            )

    return global_min_max

print(find_min_max([3, 2, 5, 1, 2, 4]))
print(find_min_max([20, 2, 5, 10, 2, 4, -20]))

MinMax(min=1, max=5)
MinMax(min=-20, max=20)


$O(n)$ time and $O(1)$ space complexity

### 11.8: Find the Kth Largest Element

In [18]:
A = [5, 4, 10, 1, 3, 11, 12, 0]

def partition_around_pivot(A: List[int], pivot_idx: int) -> None:

    pivot_value = A[pivot_idx]
    left, right = 0, len(A) - 1

    # swap pivot with rightmost value
    A[pivot_idx], A[right] = A[right], A[pivot_idx]

    new_pivot_idx = 0

    for i in range(len(A) - 1):
        if A[i] > pivot_value:
            A[i], A[new_pivot_idx] = A[new_pivot_idx], A[i]
            new_pivot_idx += 1
    A[new_pivot_idx], A[right] = A[right], A[new_pivot_idx]

partition_around_pivot(A, 2)
print(A)

[11, 12, 10, 1, 3, 5, 4, 0]


In [19]:
def search_kth_largest(A: List[int], k: int) -> int:
    ''' 
    if rearrange elements around a pivot, then if there are
    k-1 entries to the left of the pivot 
    (where left half holds values greater than pivot),
    then the pivot is the kth largest

    Note: only need kth largest not k largest
    '''
    def partition_around_pivot(left: int, right: int) -> int:

        pivot_idx = random.randint(left, right)
        pivot_value = A[pivot_idx]
        print(pivot_value)
        
        # swap pivot with rightmost value
        A[pivot_idx], A[right] = A[right], A[pivot_idx]

        new_pivot_idx = left

        for i in range(left, right):
            if A[i] > pivot_value:
                A[i], A[new_pivot_idx] = A[new_pivot_idx], A[i]
                new_pivot_idx += 1
        A[new_pivot_idx], A[right] = A[right], A[new_pivot_idx]

        return new_pivot_idx
    
    left, right = 0, len(A) - 1
    while left <= right:
        new_pivot_index = partition_around_pivot(left, right)
        print(A)
        if new_pivot_index == k - 1:      # pivot is kth largest
            return A[new_pivot_index]
        elif new_pivot_index > k - 1:     # kth largest in lower partition
            right = new_pivot_index - 1
        else:
            left = new_pivot_index + 1    # kth largest in upper partition
        
A = [5, 4, 10, 1, 3, 11, 12, 0]
search_kth_largest(A, 4)

12
[12, 4, 10, 1, 3, 11, 0, 5]
5
[12, 10, 11, 5, 3, 4, 0, 1]


5

### 11.9: Finding the Missing IP Address

### 11.10: Find the Missing and Duplicate Elements

#### Variant 11.10.A: Find Missing Element
If an array contains $n-1$ distinct integers, each between 0 and $n-1$ inclusive, then it must be the case that exactly one number between 0 and $n-1$ is absent

In [23]:
def find_missing_element(A: List[int]) -> int:
    array_sum = sum(A)
    n = len(A) + 1  # accounts for missing value
    sum_0_n = 0.5 * (n - 1) * n

    return sum_0_n - array_sum

A = list(range(100))
print(A.pop(25))
find_missing_element(A)

25


25.0

$O(n)$ time and $O(1)$ space complexity

#### Variant 11.10.B: Find Duplicate Element
If an array contains $n+1$ integers, each between 0 and $n-1$ inclusive, with exactly one element appearing twice

In [26]:
def find_duplicate_element(A: List[int]) -> int:
    array_sum = sum(A)
    n = len(A) - 1  # accounts for missing value
    sum_0_n = 0.5 * (n - 1) * n

    return array_sum - sum_0_n

A = list(range(100))
A.append(25)
find_duplicate_element(A)

25.0

$O(n)$ time and $O(1)$ space complexity